In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df = pd.read_csv('pompeia.csv', sep = ';', usecols= ['Código','Nascimento','Sexo','Consultor','Professor','Último Status','Modalidade'])
new_col_names = {'Código':'Código de cliente'}
df = df.rename(new_col_names, axis = 'columns')

In [3]:
df.head(1)

,Código de cliente,Nascimento,Sexo,Consultor,Professor,Último Status,Modalidade
0,99955264,11/04/2020,Masculino,FERNANDA LUQUE,-,30/09/2021 05:15,-


In [4]:
#Dividir a coluna do ultimo status em duas novas colunas no dataframe original, exluindo a coluna original
df[['data última presença', 'hora última presença']] = df["Último Status"].str.split(' ', expand = True)
df.drop(columns =["Último Status"], inplace = True)

In [5]:
# Convertendo strings para date e time
df['Nascimento'] = pd.to_datetime(df['Nascimento'], format="%d/%m/%Y")
df['data última presença'] = pd.to_datetime(df['data última presença'], format="%d/%m/%Y")
df['hora última presença'] = pd.to_datetime(df['hora última presença'], format="%H:%M")

In [6]:
#O cliente sedentário é o primeiro registro na ordenação por data do último status
cliente_sedentario = df[['Código de cliente','data última presença']].sort_values(by=['data última presença'])
cliente_sedentario = cliente_sedentario.head(1)

In [7]:
#1. Qual é o cliente que não vai a mais tempo?
cliente_sedentario

,Código de cliente,data última presença
464,12019,2021-01-07


In [8]:
consultores = df[['Consultor']].groupby('Consultor').agg(Total=('Consultor','count')).sort_values(by='Total', ascending=False)
consultores.head(7)

,Total
Consultor,
DANDARA MORERIO,301
FERNANDA LUQUE,292
JENIFFER MENEZES,274
LAIS VASQUES PIMENTA,146
SANDRA DANTAS,83
HERCILIA DA COSTA GOES DE ANDRADE,5
-,2


In [9]:
#2. O consultor que tem mais clientes
consultor = consultores.head(1)
consultor

,Total
Consultor,
DANDARA MORERIO,301


In [10]:
correcao = {'-':'SEM PROFESSOR'}
df['Professor'].replace(correcao, inplace= True)
professores = df[['Professor']].groupby('Professor').agg(Total=('Professor','count')).sort_values(by='Total', ascending=False)
professores.head(8)

,Total
Professor,
SEM PROFESSOR,984
ANA PAULA NASCIMENTO,55
BEATRIZ FERNANDES FROTA,19
DIEGO PEREIRA,14
FERNADO SOARES,12
YAGO OLIVEIRA,12
RENATO FIGUEIREDO,6
ANDRÉ RICARDO DA COSTA GARNECHO,1


In [ ]:
#3. Professor que tem mais alunos
professor = professores[1:2]
professor

In [80]:
#Corrigir 1 registro específico no DF
correcao = {'FUNCIONAL + MUSCULAÇÃO, NATAÇÃO':'FUNCIONAL + MUSCULAÇÃO + NATAÇÃO'}
df['Modalidade'].replace(correcao, inplace= True)
#Cria um DF limpo
temp = pd.DataFrame()
#Separa os registros em colunas diferentes
#temp[['Código de cliente']] = df['Código de cliente']
#temp[['sexo']] = df['Sexo']
#correcao = {'-':'NÃO PREENCHIDO','':'NÃO PREENCHIDO'}
#temp['sexo'].replace(correcao, inplace= True)
temp[['modalidade1','modalidade2','modalidade3']] = df['Modalidade'].str.split('+', expand = True)
#Padronização dos registros no dataframe modalidade
correcao = {'MUSCULAÇÃO ': 'MUSCULAÇÃO',' MUSCULAÇÃO':'MUSCULAÇÃO',' MUSCULAÇÃO ':'MUSCULAÇÃO','FUNCIONAL ':'FUNCIONAL',' FUNCIONAL':'FUNCIONAL','NATAÇÃO ':'NATAÇÃO',' NATAÇÃO':'NATAÇÃO','-':'PLANO COMPLETO'}
temp.replace(correcao, inplace= True)

In [81]:
temp

,modalidade1,modalidade2,modalidade3
0,PLANO COMPLETO,None,None
1,PLANO COMPLETO,None,None
2,PLANO COMPLETO,None,None
3,FUNCIONAL,MUSCULAÇÃO,None
4,FUNCIONAL,MUSCULAÇÃO,None
...,...,...,...
1098,PLANO COMPLETO,None,None
1099,PLANO COMPLETO,None,None
1100,PLANO COMPLETO,None,None
1101,PLANO COMPLETO,None,None


In [99]:
modalidades = pd.DataFrame()
modalidades['Total'] = pd.concat([temp['modalidade1'],temp['modalidade2'],temp['modalidade3']], ignore_index=True)
modalidades.head(5)

,Total
0,PLANO COMPLETO
1,PLANO COMPLETO
2,PLANO COMPLETO
3,FUNCIONAL
4,FUNCIONAL


In [102]:
#4. Qual é a modalidade mais consumida
modalidades = pd.DataFrame(modalidades['Total'].value_counts())
modalidades

,Total
PLANO COMPLETO,772
FUNCIONAL,116
NATAÇÃO INFANTIL,114
NATAÇÃO,89
MUSCULAÇÃO,85
PILATES,44
GINÁSTICA,1


In [105]:
modalidades[0:1]

,Total
PLANO COMPLETO,772
